![ALT_TEXT_FOR_SCREEN_READERS](./header.png)

# Exercise 4 Retrieval Augmented Generation

The goal of this exercise is to build a chatbot demo which allows you to talk about the content of documents. The method behind this exercise is called retrieval augmented generation (RAG).
The detailed tasks in this exercise are:
- install a local large language model using the application LM Studio
- setup a new environment with the required packages
- implement a simple chatbot using llama-index
- test the chatbot on a specific technical document

Sources for llama-index and local LLM [1] using LM Studio [2] and a small LLM [3]:
- [1] [https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/?h=embedding+model](llama-index)
- [2] [https://lmstudio.ai/](https://lmstudio.ai/)
- [3] [https://huggingface.co/models](https://huggingface.co/models)

# Considerations

- Read the tutorials carefully, especially [1]
- Install LM Studio
- Install additional software packages into the environment by uncommenting the pip install commands one time
- Select a model based on your memory size of the laptop
- This is less a coding example, rather just the integration with a local LLM

# Requirements

- R0: Install the required packages using the pip commands
- R1: Install the LM Studio
- R2: Find a model which is running on your machine
- R3: Start the server for the model in LM Studio
- R4: Connect the server to the notebook
- R5: Run the code parts until the first query
- R6: Improve your query according to the slides learned in the class


In [ ]:
#!pip install ipywidgets

In [1]:
#!pip install llama-index

In [ ]:
#!pip install pip install llama-index-llms-openai-like

In [2]:
#!pip install llama-index-llms-ollama

In [4]:
#!pip install llama-index-embeddings-huggingface

# Code Snipplet

In [5]:
from llama_index.llms.ollama import Ollama

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [8]:
from llama_index.core.embeddings import resolve_embed_model

In [12]:
from llama_index.core.llms import ChatMessage

## Prepare LLM

In [9]:
llm = Ollama(model="llama3.1:latest", request_timeout=120.0)

In [10]:
# test LLM complete interface
resp = llm.complete("Who is Paul Graham?")

In [11]:
print(resp)

Paul Graham is a British-American computer scientist, entrepreneur, and writer. He is best known as the co-founder of Y Combinator, a startup accelerator that has funded many successful technology companies.

Graham was born in 1964 in England and studied mathematics at University College London. He then moved to the United States and earned a Ph.D. in computer science from Harvard University. In the early days of the web, Graham worked as a programmer and entrepreneur, co-founding several startups, including Viaweb, an online store that was later sold to Yahoo! for $49 million.

In 2005, Graham co-founded Y Combinator with Robert Tappan Morris (son of computer scientist Bob Morris) and Jessica Livingston. The accelerator program provides funding, mentorship, and resources to early-stage startups in exchange for a small equity stake. Since its inception, Y Combinator has invested in over 2,000 companies, including Airbnb, Dropbox, Reddit, and Cruise, among many others.

Graham is also 

In [13]:
# test chat interface
messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [14]:
print(resp)

assistant: Me hearty! Me name be Captain Calico Jack "Blackheart" McSquirt, the most feared and infamous pirate to ever sail the Seven Seas! Me crew calls me "Cal", but only after they've had a few too many grogs. Arrr!

Me nickname "Blackheart" don't just mean I'm as black as coal, savvy? It means I've got a heart o' gold... for plunderin', pillagin', and generally causin' chaos on the high seas! Me motto be: "Plunder or perish!" And that's exactly what me and me crew do best!

Now, what can I do fer ye, matey? Ye lost at sea, or just lookin' fer a swashbucklin' good time?


In [15]:
Settings.llm = llm

## Prepare Embedding Model

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en",cache_folder='./')

## Generate Vector Store Content

In [ ]:
documents = SimpleDirectoryReader("documents").load_data()

In [29]:
index = VectorStoreIndex.from_documents(documents,chunk_size=128, chunk_overlap=64)

## Setup Query Engine

In [30]:
query_engine = index.as_query_engine(streaming=False, verbose=True, similarity_top_k=2)

In [35]:
prompts_dict = query_engine.get_prompts()
print(list(prompts_dict.keys()))

['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template']


In [37]:
prompts_dict['response_synthesizer:text_qa_template']

SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x16b6c2020>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an expert

## First Query

In [31]:
response = query_engine.query("What is the maximal output power of the inverter?")

In [32]:
print(response)

366 VA


# Improved Query

In [34]:
response = query_engine.query("What are the FCC compiance rules? Summarize the context information and generate a short answer.")
print(response)


The FCC compliance rules are not mentioned in the context information, so I cannot generate the requested summary.


In [39]:
response = query_engine.query("Please extract the corporate contact information.")
print(response)


The corporate headquarters contact information is not explicitly mentioned in the context information, so I cannot extract the requested data from the context.
